In [384]:
K = 5; 
D = 1;

n = 2*K
d = 2*D

sigma = [1]; #pick a subset of {1,2}.

In [385]:
using HomotopyContinuation
using IterTools
using LinearAlgebra
using Combinatorics


# Load the Fermi Dirac types
include("Algebra-of-spin-restricted-CC/SpinRestrictedHomotopy/operators.jl")


# Load pretty print for operators
include("Algebra-of-spin-restricted-CC/SpinRestrictedHomotopy/operator_term_enhanced.jl")


In [386]:
function multisets(xs::AbstractVector{T}, k::Int)::Vector{Vector{T}} where T
    n = length(xs)

    # edge cases
    if k < 0
        throw(ArgumentError("k must be nonnegative"))
    elseif k == 0
        return [T[]]                 # one empty multiset
    elseif n == 0
        return Vector{Vector{T}}()   # no multisets if there are no items
    end

    result = Vector{Vector{T}}()
    buf = Vector{T}(undef, k)        # buffer to build each multiset in-place

    function backtrack(pos::Int, start::Int)
        if pos > k
            push!(result, copy(buf)) # save a copy of the current multiset
            return
        end

        for i in start:n
            buf[pos] = xs[i]
            backtrack(pos + 1, i)    # i (not i+1) → allow repetitions
        end
    end

    backtrack(1, 1)
    return result
end

multisets (generic function with 1 method)

In [387]:
X = [Op(a, true, true)*Op(i, false, true) + Op(a, true, false)*Op(i, false, false) for i in 1:K, a in 1:K]

5×5 Matrix{FD{Int64}}:
 a_{2}^†a_{2} + a_{1}^†a_{1}   …  a_{10}^†a_{2} + a_{9}^†a_{1}
 a_{2}^†a_{4} + a_{1}^†a_{3}      a_{10}^†a_{4} + a_{9}^†a_{3}
 a_{2}^†a_{6} + a_{1}^†a_{5}      a_{10}^†a_{6} + a_{9}^†a_{5}
 a_{2}^†a_{8} + a_{1}^†a_{7}      a_{10}^†a_{8} + a_{9}^†a_{7}
 a_{2}^†a_{10} + a_{1}^†a_{9}     a_{10}^†a_{10} + a_{9}^†a_{9}

In [388]:
basis = collect(subsets(1:n,d));

function base_lt(x, y, r)
    return length(x ∩ r) > length(y ∩ r)
end;

sort!(basis, lt=(x,y) -> base_lt(x,y, Set(1:d)))

trunc_bas = filter(b -> length(setdiff(b, Set(1:d))) in [0;sigma], basis)

17-element Vector{Vector{Int64}}:
 [1, 2]
 [1, 3]
 [1, 4]
 [1, 5]
 [1, 6]
 [1, 7]
 [1, 8]
 [1, 9]
 [1, 10]
 [2, 3]
 [2, 4]
 [2, 5]
 [2, 6]
 [2, 7]
 [2, 8]
 [2, 9]
 [2, 10]

In [389]:
T = 0
vars = Vector{Variable}([])

ex_bas = vcat(collect(product(1:D, 1:(K - D)))...)

for k in sigma
    for M in multisets(ex_bas, k)
        I  = [m[1] for m in M];
        B = [m[2] + D for m in M];
        Is = parse(Int64, string(I...));
        Bs = parse(Int64, string(B...));
        
        var = Variable(:t, Is, Bs)
        push!(vars,var)

        cons = k == 2 && M[1] == M[2] ? 2 : 1;

        T += (1//cons)*var*prod([X[I[l],B[l]] for l in 1:k])
    end
end

T

t₁₋₅a_{10}^†a_{2} + t₁₋₂a_{4}^†a_{2} + t₁₋₂a_{3}^†a_{1} + t₁₋₃a_{6}^†a_{2} + t₁₋₄a_{7}^†a_{1} + t₁₋₅a_{9}^†a_{1} + t₁₋₄a_{8}^†a_{2} + t₁₋₃a_{5}^†a_{1}

In [390]:
length(vars)

4

In [391]:
expT = 1 + sum([(1//factorial(k))*T^k for k in 1:d]);

In [392]:
cb = with_limited_threads(1) do
      cb_local = [[1;zeros(length(trunc_bas) - 1, 1)][:,1]]

      for k in sigma
          for M in multisets(ex_bas, k)
              cons = k == 2 && M[1] == M[2] ? 2 : 1;
              vec = (1//cons)*matrixrep(prod([X[m[1],m[2] + D] for m in M]), trunc_bas, [basis[1]])[:,1]
              push!(cb_local, vec)
          end
      end

      cb_local
  end


cb = Matrix{Int64}(hcat(cb...));

Q, R = qr(cb)
cb = Matrix(Q);

In [393]:
@var h[1:K, 1:K], v[1:K,1:K,1:K,1:K], lambda;

In [394]:
h0 = [h[min(i,j),max(i,j)] for i in 1:K, j in 1:K];

h_params = [h0[i,j] for i in 1:K for j in i:K];

In [395]:
w = Array{Expression}(undef, K, K, K, K);


for ((i,j),(k,l)) in multisets(multisets(1:K,2), 2)
    w[i,j,k,l] = w[k,l,i,j] = w[j,i,l,k] = w[l,k,j,i] = v[i,j,k,l];
    w[j,i,k,l] = w[l,k,i,j] = w[i,j,l,k] = w[k,l,j,i] = v[i,j,k,l];     
end


v_params = [v[L[1],L[2], M[1], M[2]] for (L,M) in multisets(multisets(1:K,2), 2)]

params = Vector{Variable}([h_params;v_params])

(length(h_params), length(v_params))

(15, 120)

In [396]:
H0 = sum([h0[i,j]*X[i,j] for i in 1:K for j in 1:K]);

Vops = sum([w[i,j,k,l]*(Op(i,true, true)*Op(k,true, true)*Op(l, false, true)*Op(j,false, true) + Op(i,true, true)*Op(k,true, false)*Op(l, false, false)*Op(j,false, true) + Op(i,true, false)*Op(k,true, true)*Op(l, false, true)*Op(j,false, false) + Op(i,true, false)*Op(k,true, false)*Op(l, false, false)*Op(j,false, false))
    for i in 1:K for j in 1:K for k in 1:K for l in 1:K]);

H = H0 + (1/2)*Vops;

In [397]:
bigCC = (H - lambda)*expT;

CCeqs = with_limited_threads(1) do
    (transpose(cb)*matrixrep(bigCC, trunc_bas, [basis[1]]))[:,1];
end

F = System(CCeqs, variables = [lambda;vars], parameters=params);

In [398]:
start_sol = rand(ComplexF64, length(vars) + 1);
#start_sol = rand(numvars);

param_sys = evaluate(F, start_sol);

In [399]:
# For real matrx

#A = [Float64.(differentiate(poly, params)) for poly in param_sys]

#A = reduce(hcat, A)';

# For complex matrix

A = [differentiate(poly, param) for poly in param_sys, param in params];

A = convert(Array{ComplexF64}, A);

In [400]:
F_param = System(param_sys, variables = params);
b = evaluate(F_param, zeros(length(params)));

In [401]:
nullsp = nullspace(A);
xh = A\(-b);

# Real matrix
#start_params = nullsp*rand(size(nullsp)[2]) + xh;

#Complex matrix
start_params = nullsp*rand(ComplexF64, size(nullsp)[2]) + xh;

evaluate(F_param, start_params)

5-element Vector{ComplexF64}:
  5.551115123125783e-16 + 0.0im
 -5.765183246729486e-16 - 5.78971594139642e-16im
 -2.453269466693398e-16 + 4.592520441650041e-15im
 -4.955604322720665e-16 + 2.89485797069821e-16im
 -8.831770080096234e-16 - 2.3551386880256624e-16im

In [402]:
result = monodromy_solve(F,start_sol, start_params, threading=true, catch_interrupt = true)

MonodromyResult
• return_code → :heuristic_stop
• 31 solutions
• 217 tracked loops
• random_seed → 0x71a4e3db

In [403]:
sig = ""
if 1 in sigma
    sig = sig*"S"
end
if 2 in sigma 
     sig = sig*"D"
end

In [404]:
write_solutions("Algebra-of-spin-restricted-CC/SpinRestrictedHomotopy/MonodromySolutions/$(D)e$(K)oCC"*sig*"sols.txt", solutions(result))
write_parameters("Algebra-of-spin-restricted-CC/SpinRestrictedHomotopy/MonodromySolutions/$(D)e$(K)oCC"*sig*"param.txt", parameters(result))
